1

![daRnn](daRnnChart.jpg)

decomposing the process of a daRnn (Dual-Stage Attention-Based Recurrent Neural Network) model step by step

daRnn is a special nonlinear autoregressive network with exogenous inputs (NARX) $$\hat{Y}_{T} =F\left(Y_{1}, \cdots, Y_{T-1}, {X}_{1}, \cdots, {X}_{T}\right)$$

before we start, let's get familiar with some notations to understand the shape of input data.



$$X = \begin{bmatrix}
x_1^1 & x_2^1 & ...& ... & ... & x_T^1\\
x_1^2 & x_2^2 & ...& ... & ... & x_T^2 \\
... & ... & ...& ... & ... & ...\\
... & ... & ...& x_t^k & ... & x_T^k \\
... & ... & ...& ... & ... & ...  \\
x_1^n & x_2^n & ...& ... & ... & x_T^n \\
\end{bmatrix} = (x^1, x^2, ..., x^k, ... , x^n)' = (x_1, x_2, ... , x_t, ..., x_T)$$

in which $n$ is the number of explainary variables and $T$ is the length of the time series.



**step 1**: initialize the hidden state $s_t$, $h_t$. their shape are both $m$, $t=1$

**step 2**: calculate the similarity of $h_{t-1}$, $s_{t-1}$ with $x^k$ by the following equation(Eq. #8 in the paper):

$$
e_{t}^{k}=v_{e}' \tanh \left(W_{e}\left[h_{t-1} ; s_{t-1}\right]+U_{e} x^{k}\right), k=1,2,...,n
$$ 

while coding, we input $X$, so we can do this for all $k$ from $1$ to $n$ at the same time thanks to matrix opertion. That is $(e_1^1, e_1^2, ... , e_1^n)'$. Then we softmax it (Eq. #9 in the paper) to get $(\alpha_1^1, \alpha_1^2, ... , \alpha_1^n)'$, so as to undate the original $x_t$ (the $t^{th}$ column in $X$) with:

$$
\tilde{x}_{t}=\left(\alpha_{t}^{1} x_{t}^{1}, \alpha_{t}^{2} x_{t}^{2}, \cdots, \alpha_{t}^{n} x_{t}^{n}\right)'
$$

**step 3**: pass $\tilde{x}_{t}$ to a LSTM cell, together with $h_{t-1}$, $s_{t-1}$, to get $h_t$ and $s_t$, the hidden state of the next time. the detail of how LSTM works is as follows:

$$
\begin{aligned}
{f}_{t} &=\sigma\left({W}_{f}\left[{h}_{t-1} ; {x}_{t}\right]+{b}_{f}\right) \\
{i}_{t} &=\sigma\left({W}_{i}\left[{h}_{t-1} ; {x}_{t}\right]+{b}_{i}\right) \\
{o}_{t} &=\sigma\left({W}_{o}\left[{h}_{t-1} ; {x}_{t}\right]+{b}_{o}\right) \\
{s}_{t}={f}_{t} \odot {s}_{t-1} &+{i}_{t} \odot \tanh \left({W}_{s}\left[{h}_{t-1} ; {x}_{t}\right]+{b}_{s}\right) \\
{h}_{t} &={o}_{t} \odot \tanh \left({s}_{t}\right)
\end{aligned}
$$

**step 4**: return to step 2 untill $t=T$

*now we already have $h_1, h_2, ..., h_T$. it's time to apply another attnetion layer. if we put these $h_t$ together, which is $(h_1, h_2, ..., h_T)$, it's not difficult to find they have a very similar shape as $X = (x_1, x_2, ..., x_T)$.*

**step 5**: generate the initial hidden state $s_{t-1}'$, $d_{t-1}$, $t=1$

**step 6**: here is the tricky point of daRnn, we will not calculate the similarity of hidden state and some rows of $(h_1, h_2, ..., h_T)$ like what we do in the first attention layer ($x^k$ is a row of $X$). instead, we calculate the similarity of hidden state $s_{t-1}'$, $d_{t-1}$ and $h_1, h_2, ..., h_T$ respectively by using Eq #12 in the paper:
$$
l_{t}^{i}=v_{d}‘ \tanh \left({W}_{d}\left[{d}_{t-1} ; {s}_{t-1}^{\prime}\right]+{U}_{d} {h}_{i}\right), i=1,2,...,T
$$

like before, we softmax $l_t^1, l_t^2, ..., l_t^T$, the output is $\beta_t^1, \beta_t^2, ..., \beta_t^T$

**step 7**: $c_t = \sum_{i=1}^T h_i \beta_t^i$

**step 8**: pass $c_t$ to a LSTM cell, together with $s_{t-1}'$, $d_{t-1}$, to get $s'_t$ and $d_t$, the hidden state of the next time.

**step 9**: return to step 6 until $t=T$

**step 10**: subtract the final step output $d_T, c_T$

$$
\begin{aligned}
\hat{y}_{T} &=F\left(y_{1}, \cdots, y_{T-1}, {x}_{1}, \cdots, {x}_{T}\right) \\
&={v}_{y}^{\top}\left({W}_{y}\left[{d}_{T} ; {c}_{T}\right]+{b}_{w}\right)+b_{v}
\end{aligned}
$$